# Deployment and Interface Demo
This notebook demonstrates how to verify Docker containers, create a Gradio interface, and prepare for deployment.

## 1. Install and Import Dependencies

In [ ]:
%pip install docker gradio scikit-learn pandas
import docker
import gradio as gr
import pandas as pd
import pickle
import os

## 2. Verify Docker Container Status

In [ ]:
try:
    client = docker.from_env()
    containers = client.containers.list(all=True)
    print("Active and Stopped Containers:")
    for container in containers:
        print(f"ID: {container.short_id}, Name: {container.name}, Status: {container.status}")
except Exception as e:
    print(f"Error connecting to Docker: {e}")
    print("Trying subprocess...")
    import subprocess
    result = subprocess.run(["docker", "ps", "-a"], capture_output=True, text=True)
    print(result.stdout)

## 3. Load Pre-trained Model
For this demo, we will create a simple dummy model if the actual one is not easily loadable without complex dependencies.

In [ ]:
# Create a dummy model for demonstration purposes
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification

# Generate dummy data
X, y = make_classification(n_samples=100, n_features=9, random_state=42)
model = LogisticRegression()
model.fit(X, y)

print("Dummy model created and trained.")

## 4. Create Prediction Interface with Gradio

In [ ]:
def predict_potability(ph, hardness, solids, chloramines, sulfate, conductivity, organic_carbon, trihalomethanes, turbidity):
    # Prepare input array
    input_data = [[ph, hardness, solids, chloramines, sulfate, conductivity, organic_carbon, trihalomethanes, turbidity]]
    prediction = model.predict(input_data)[0]
    return "Potable" if prediction == 1 else "Not Potable"

iface = gr.Interface(
    fn=predict_potability,
    inputs=[
        gr.Number(label="pH"),
        gr.Number(label="Hardness"),
        gr.Number(label="Solids"),
        gr.Number(label="Chloramines"),
        gr.Number(label="Sulfate"),
        gr.Number(label="Conductivity"),
        gr.Number(label="Organic Carbon"),
        gr.Number(label="Trihalomethanes"),
        gr.Number(label="Turbidity")
    ],
    outputs="text",
    title="Water Potability Predictor",
    description="Enter water quality metrics to predict potability."
)

# Launch the interface (commented out to prevent blocking in some environments, uncomment to run)
# iface.launch()
print("Gradio interface created. Run iface.launch() to view.")

## 5. Prepare for Deployment: Dockerfile Creation

In [ ]:
dockerfile_content = """
FROM python:3.9-slim

WORKDIR /app

COPY requirements.txt .
RUN pip install -r requirements.txt

COPY . .

CMD ["python", "app.py"]
"""

with open("Dockerfile_gradio", "w") as f:
    f.write(dockerfile_content)

print("Dockerfile_gradio created.")

## 6. Build and Run Deployment Container

In [ ]:
# Build the image (This is a placeholder command, actual build might take time)
print("To build the image, run:")
print("docker build -t water-gradio-app -f Dockerfile_gradio .")

# Run the container
print("\nTo run the container:")
print("docker run -p 7860:7860 water-gradio-app")